# insightface face recognition

In [107]:
import os
import pickle
import numpy as np
from PIL import Image
from typing import List
from tqdm import tqdm

from insightface.app import FaceAnalysis
from sklearn.neighbors import NearestNeighbors

In [ ]:
app = FaceAnalysis(name = "antelope") # model 저장 위치 Users/user/.insightface/models/antelope

In [109]:
app.prepare(ctx_id=0, det_size=(640, 640))

set det-size: (640, 640)


## 사진 이미지 임베딩 출력

In [115]:
import cv2
img = cv2.imread('data/img4.jpeg')
img_emb_results = app.get(np.asarray(img))
img_emb = img_emb_results[0].embedding
img_emb.shape

(512,)

In [116]:
img_emb_results

[Face(bbox=array([192.98572 ,  94.814896, 394.8215  , 389.65363 ], dtype=float32), kps=array([[276.46716, 229.52318],
        [362.5609 , 217.9119 ],
        [345.95984, 275.8536 ],
        [301.20508, 333.75522],
        [365.43045, 323.6345 ]], dtype=float32), det_score=0.87492967, embedding=array([-0.21668223,  0.5687366 , -0.565464  ,  0.7164002 ,  0.5212341 ,
        -1.3351742 , -0.5303242 , -1.2011125 ,  0.17439117, -0.04537246,
        -0.950419  ,  0.7850304 ,  1.6782633 , -0.16926806, -0.7851038 ,
        -0.45390096, -1.8479828 , -0.56671154, -0.8060133 ,  0.03830765,
         0.06272234, -0.5534952 , -1.732264  , -0.03220292, -0.71493   ,
        -0.08578123,  1.4182367 , -0.32269585,  0.19190781, -1.1863693 ,
         1.4491197 ,  0.02314605, -1.384956  , -0.6424856 , -0.5770625 ,
        -1.6225928 , -0.15051079, -0.32042715,  1.2978078 ,  1.5463716 ,
        -0.29798827, -0.20263687,  0.08159092,  0.47062653, -0.80447614,
         0.78754526,  1.7421468 , -0.9612456 , -0

In [117]:
img_emb_results[0].bbox

array([192.98572 ,  94.814896, 394.8215  , 389.65363 ], dtype=float32)

In [131]:
# 이미지 bbox 출력
cv2.rectangle(img, 
               (int(img_emb_results[0].bbox[0]), int(img_emb_results[0].bbox[1])), 
               (int(img_emb_results[0].bbox[2]), int(img_emb_results[0].bbox[3])), 
               (255, 255, 255), 1)

array([[[210, 214, 203],
        [210, 214, 203],
        [209, 213, 202],
        ...,
        [209, 214, 205],
        [209, 214, 205],
        [209, 214, 205]],

       [[210, 214, 203],
        [210, 214, 203],
        [209, 213, 202],
        ...,
        [209, 214, 205],
        [209, 214, 205],
        [209, 214, 205]],

       [[210, 214, 203],
        [210, 214, 203],
        [210, 214, 203],
        ...,
        [209, 214, 205],
        [209, 214, 205],
        [209, 214, 205]],

       ...,

       [[ 47,  41,  28],
        [ 48,  42,  29],
        [ 49,  43,  30],
        ...,
        [ 63,  72,  76],
        [ 61,  69,  76],
        [ 59,  67,  74]],

       [[ 47,  41,  28],
        [ 47,  41,  28],
        [ 47,  41,  28],
        ...,
        [ 57,  63,  68],
        [ 58,  67,  71],
        [ 59,  68,  72]],

       [[ 47,  41,  28],
        [ 47,  41,  28],
        [ 46,  40,  27],
        ...,
        [ 51,  58,  61],
        [ 58,  67,  71],
        [ 64,  73,  77]]

In [ ]:
# 이미지 출력
import matplotlib.pyplot as plt
plt.figure(figsize= (10, 10))
plt.imshow(img[:, :, ::-1])
plt.show()

## face recognition

In [74]:
# Fixing the file extensions
YALE_DIR = "data/imgdata" # 사진 이미지 위치
files = os.listdir(YALE_DIR)[0:] # file list
for i, img in enumerate(files): # gif형태로 변경
    # print("original name: ", img)
    new_ext_name = "_".join(img.split(".")) + ".gif"
    # print("new name: ",  new_ext_name)
    os.rename(os.path.join(YALE_DIR, img), os.path.join(YALE_DIR, new_ext_name))

In [75]:
# 임베딩 생성 함수
def generate_embs(img_fpaths: List[str]):
    embs_set = list()
    embs_label = list()

    for img_fpath in img_fpaths:  
                    
        # read grayscale img
        img = Image.open(os.path.join(YALE_DIR, img_fpath)) 
        img_arr = np.asarray(img)  
        
        # convert grayscale to rgb
        im = Image.fromarray((img_arr * 255).astype(np.uint8))
        rgb_arr = np.asarray(im.convert('RGB'))       
       
        # generate Insightface embedding
        res = app.get(rgb_arr)          
        # append emb to the eval set
        embs_set.append(res)          
        # append label to eval_label set
        embs_label.append(img_fpath.split("_")[0])          

    return embs_set, embs_label

In [76]:
files = os.listdir(YALE_DIR)
files.sort()
eval_set = list()
eval_labels = list()
IMAGES_PER_IDENTITY = 11
# 이미지 임베딩 후 eval_set, eval_labels 저장
for i in tqdm(range(1, len(files), IMAGES_PER_IDENTITY), unit_divisor=True): # ignore the README.txt file at files[0]
    
    # store eval embs and labels
    eval_set_t, eval_labels_t = generate_embs(files)
    eval_set.extend(eval_set_t)
    eval_labels.extend(eval_labels_t)

100%|██████████| 1/1 [00:00<00:00,  1.24it/s]


In [77]:
eval_labels

['img1', 'img2', 'img3']

In [78]:
# 비어있는 embedding img 제외
def filter_empty_embs(img_set: List, img_labels: List[str]):
    # filtering where insightface could not generate an embedding
    good_idx = [i for i,x in enumerate(img_set) if x]
    
    if len(good_idx) == len(img_set):
        clean_embs = [e[0].embedding for e in img_set]
        clean_labels = img_labels
        
    else:
        # filtering eval set and labels based on good idx
        clean_labels = np.array(img_labels)[good_idx]
        clean_set = np.array(img_set, dtype=object)[good_idx]
        
        # generating embs for good idx
        clean_embs = [e[0].embedding for e in clean_set]
    
    return clean_embs, clean_labels

In [79]:
evaluation_embs, evaluation_labels = filter_empty_embs(eval_set, eval_labels)

In [80]:
evaluation_embs

[array([-1.2249392 ,  1.2985175 , -1.9518539 , -0.07087362,  0.94352627,
         1.484796  , -0.11804366, -0.5776401 , -0.54042   , -0.88340306,
         0.3490669 ,  1.4866939 ,  0.8172993 , -0.2459126 ,  0.75505847,
        -1.1774052 , -1.2745379 , -0.1741676 ,  1.8583612 , -0.5331424 ,
        -0.84389645, -2.2636595 ,  1.056642  , -0.7036505 ,  0.8142494 ,
        -1.2605503 ,  0.29242676, -0.51225454, -0.8154127 , -0.44311875,
        -0.11493419, -0.26202682, -0.5842905 , -0.54583347, -1.4151465 ,
        -0.06120177,  0.49393812, -0.80674434, -1.3289928 ,  0.03412659,
        -0.34498933, -0.16267824,  1.0166115 , -1.2684789 ,  0.04436588,
        -1.0735482 ,  1.0200391 ,  0.7285003 , -0.1234091 ,  0.04226086,
         1.191946  , -1.021035  ,  0.30475467,  0.5065459 , -0.34344196,
        -0.21821097, -0.41486067, -0.81737673,  1.4492265 ,  0.6606788 ,
        -0.997141  , -1.1798224 , -1.0775921 ,  0.5606862 , -0.10281654,
        -0.5518401 , -0.74181306, -1.5542294 , -0.4

## 임베딩 거리

In [81]:
# Nearest neighbour learning method
nn = NearestNeighbors(n_neighbors=3, metric="cosine")
nn.fit(X=evaluation_embs)

# save the model to disk
#filename = 'faceID_model.pkl'
#with open(filename, 'wb') as file:
#    pickle.dump(nn, file)
    
# some time later...
# load the model from disk
# with open(filename, 'rb') as file:
#     pickle_model = pickle.load(file)

NearestNeighbors(metric='cosine', n_neighbors=3)

In [88]:
# 임베딩 비교 후 결과 출력
def print_ID_results(img_fpath: str, evaluation_labels: np.ndarray, verbose: bool = False):
    img = Image.open(img_fpath)
    img_emb = app.get(np.asarray(img))[0].embedding
    
    # get pred from KNN
    dists, inds = nn.kneighbors(X=img_emb.reshape(1,-1), n_neighbors=3, return_distance=True)
    
    # get labels of the neighbours
    pred_labels = [evaluation_labels[i] for i in inds[0]]
    
    # check if any dist is greater than 0.5, and if so, print the results
    no_of_matching_faces = np.sum([1 if d <=0.6 else 0 for d in dists[0]])
    if no_of_matching_faces > 0:
        print("Matching face(s) found in database! ")
        verbose = True
    else: 
        print("No matching face(s) not found in database!")
        
    # print labels and corresponding distances
    if verbose:
        for label, dist in zip(pred_labels, dists[0]):
            print(f"Nearest neighbours found in the database have labels {label} and is at a distance of {dist}")


In [89]:
print_ID_results("data/img2_zoom.jpeg", evaluation_labels, verbose=True)

No matching face(s) not found in database!
Nearest neighbours found in the database have labels img1 and is at a distance of 0.9024404883384705
Nearest neighbours found in the database have labels img2 and is at a distance of 0.9581493735313416
Nearest neighbours found in the database have labels img3 and is at a distance of 0.9957922101020813
